# Basic experiments with GPU programming in julia

This is roughly based on the CUArrays package introduction

In [1]:
using BenchmarkTools
using Test
using Statistics

## Element-wise product experiments

Test performance of a simple (large) vector Hadamard product

In [2]:
N=2^28
x = fill(2.0f0, N)
y = fill(3.0f0, N)
;

### Single threaded on a CPU, using broadcasting:

In [3]:
@test all(x .* y .== 6.0f0)
@benchmark z = x .* y

BenchmarkTools.Trial: 
  memory estimate:  1.00 GiB
  allocs estimate:  4
  --------------
  minimum time:     527.503 ms (0.03% GC)
  median time:      555.744 ms (4.61% GC)
  mean time:        556.384 ms (5.06% GC)
  maximum time:     591.918 ms (11.05% GC)
  --------------
  samples:          9
  evals/sample:     1

### Single threaded on a CPU, using loops:

In [4]:
function serial_mul(x, y)
    z = similar(x)
    for i in eachindex(x,y,z)
        @inbounds z[i] = x[i] * y[i]
    end
    return z
end

@test all(serial_mul(x, y) .== 6.0f0)
@benchmark z = serial_mul(x, y)

BenchmarkTools.Trial: 
  memory estimate:  1.00 GiB
  allocs estimate:  2
  --------------
  minimum time:     527.501 ms (0.03% GC)
  median time:      553.928 ms (4.61% GC)
  mean time:        555.760 ms (5.11% GC)
  maximum time:     595.746 ms (11.16% GC)
  --------------
  samples:          9
  evals/sample:     1

### Multi threaded on a CPU:

In [5]:
function threaded_mul(x, y)
    z = similar(x)
    Threads.@threads for i in eachindex(x,y,z)
        @inbounds z[i] = x[i] * y[i]
    end
    return z
end

@test all(threaded_mul(x, y) .== 6.0f0)
@benchmark z = threaded_mul(x, y)

BenchmarkTools.Trial: 
  memory estimate:  1.00 GiB
  allocs estimate:  3
  --------------
  minimum time:     88.876 ms (12.38% GC)
  median time:      122.169 ms (35.71% GC)
  mean time:        125.183 ms (33.65% GC)
  maximum time:     184.174 ms (25.04% GC)
  --------------
  samples:          40
  evals/sample:     1

### GPU with broadcasting (multithreaded and grided)

In [6]:
using CuArrays

x_gpu = cufill(2.0f0, N)
y_gpu = cufill(3.0f0, N)
;

In [7]:
function broadcast_gpu_mul(x, y)
    CuArrays.@sync z = x .* y
    return z
end

@test all(broadcast_gpu_mul(x_gpu, y_gpu) .== 6.0f0)
@benchmark z_gpu = broadcast_gpu_mul(x_gpu, y_gpu)

BenchmarkTools.Trial: 
  memory estimate:  2.27 KiB
  allocs estimate:  62
  --------------
  minimum time:     7.425 ms (0.00% GC)
  median time:      8.576 ms (0.00% GC)
  mean time:        48.540 ms (39.45% GC)
  maximum time:     292.942 ms (51.80% GC)
  --------------
  samples:          103
  evals/sample:     1

### Single threaded on GPU (custom kernel)

In [8]:
using CUDAnative

In [9]:
function serial_gpu_mul!(z, x, y)
    for i in 1:length(x)
        @inbounds z[i] = x[i] * y[i]
    end
    return nothing
end

function serial_gpu_mul(x, y)
    z = similar(x)
    CuArrays.@sync begin
        @cuda serial_gpu_mul!(z, x, y)
    end
    return z
end

@test all(serial_gpu_mul(x_gpu, y_gpu) .== 6.0f0)
@benchmark z_gpu = serial_gpu_mul(x_gpu, y_gpu)

BenchmarkTools.Trial: 
  memory estimate:  912 bytes
  allocs estimate:  34
  --------------
  minimum time:     11.969 s (0.00% GC)
  median time:      11.969 s (0.00% GC)
  mean time:        11.969 s (0.00% GC)
  maximum time:     11.969 s (0.00% GC)
  --------------
  samples:          1
  evals/sample:     1

### Multi threaded on GPU, single SM

In [10]:
function threaded_gpu_mul!(z, x, y)
    index = threadIdx().x
    stride = blockDim().x
    
    for i in index:stride:length(x)
        @inbounds z[i] = x[i] * y[i]
    end
    return nothing
end

function threaded_gpu_mul(x, y)
    z = similar(x)
    CuArrays.@sync begin
        @cuda threads=1024 threaded_gpu_mul!(z, x, y)
    end
    return z
end

@test all(threaded_gpu_mul(x_gpu, y_gpu) .== 6.0f0)
@benchmark z_gpu = threaded_gpu_mul(x_gpu, y_gpu)

BenchmarkTools.Trial: 
  memory estimate:  944 bytes
  allocs estimate:  36
  --------------
  minimum time:     87.583 ms (0.00% GC)
  median time:      89.332 ms (0.00% GC)
  mean time:        128.703 ms (14.55% GC)
  maximum time:     381.845 ms (40.29% GC)
  --------------
  samples:          39
  evals/sample:     1

### Multithreaded and grided on GPU

In [11]:
function grided_gpu_mul!(z, x, y)
    index = threadIdx().x + (blockIdx().x - 1)*blockDim().x
    stride = blockDim().x * gridDim().x
    
    for i in index:stride:length(x)
        @inbounds z[i] = x[i] * y[i]
    end
    return nothing
end

function grided_gpu_mul(x, y)
    z = similar(x)
    numthreads = 64
    numblocks = ceil(Int, length(x)/numthreads)
    CuArrays.@sync begin
        @cuda threads=numthreads blocks=numblocks grided_gpu_mul!(z, x, y)
    end
    return z
end

grided_gpu_mul (generic function with 1 method)

In [12]:
@test all(grided_gpu_mul(x_gpu, y_gpu) .== 6.0f0)
@benchmark z_gpu = grided_gpu_mul(x_gpu, y_gpu)

BenchmarkTools.Trial: 
  memory estimate:  960 bytes
  allocs estimate:  36
  --------------
  minimum time:     7.323 ms (0.00% GC)
  median time:      8.175 ms (0.00% GC)
  mean time:        47.277 ms (37.82% GC)
  maximum time:     259.889 ms (44.75% GC)
  --------------
  samples:          111
  evals/sample:     1

### Multiple GPUs

In [13]:
numgpus = 10
n = ceil(Int, N/numgpus)

function devicefill(dev, value, shape)
    device!(dev) do
        return cufill(value, shape)
    end
end

x_gpus = [devicefill(dev, 2.0f0, n) for dev in 0:(numgpus-1)]
y_gpus = [devicefill(dev, 3.0f0, n) for dev in 0:(numgpus-1)]
z_gpus = [devicefill(dev, 2.0f0, n) for dev in 0:(numgpus-1)]
;

In [14]:
function multi_gpu_mul!(z_gpus, x_gpus, y_gpus)
    function kernel!(z, x, y)
        index = threadIdx().x + (blockIdx().x - 1)*blockDim().x
        stride = blockDim().x * gridDim().x
    
        for i in index:stride:length(x)
            @inbounds z[i] = x[i] * y[i]
        end
        return nothing
    end
    
    # launch each kernel
    for i in 1:numgpus
        device!(i-1) do
            numthreads = 128
            numblocks = ceil(Int, length(x_gpus[i])/numthreads)
            @cuda threads=numthreads blocks=numblocks kernel!(z_gpus[i], x_gpus[i], y_gpus[i])
        end
    end
    # resynchronize each device (for timing purposes)
    for i in 1:numgpus
        device!(i-1) do
            CuArrays.@sync 1
        end
    end
    return z_gpus
end

@test all(vcat((Array.(multi_gpu_mul!(z_gpus, x_gpus, y_gpus)))...) .== 6.0f0)
CuArrays.@time multi_gpu_mul!(z_gpus, x_gpus, y_gpus)
@benchmark multi_gpu_mul!(z_gpus, x_gpus, y_gpus)

  0.001097 seconds (252 CPU allocations: 6.781 KiB)


BenchmarkTools.Trial: 
  memory estimate:  6.78 KiB
  allocs estimate:  252
  --------------
  minimum time:     896.695 μs (0.00% GC)
  median time:      991.684 μs (0.00% GC)
  mean time:        1.042 ms (0.44% GC)
  maximum time:     35.347 ms (61.21% GC)
  --------------
  samples:          4742
  evals/sample:     1

## Tensor contraction experiment

Calculate a simple tensor contraction, i.e.

$$z_{i,j,k} = \sum_{l} x_{l,i,j} y_{l, k}$$

In [15]:
using Random
rng = MersenneTwister(1842)

t1 = randn(rng, Float32, (200, 600, 300))
t2 = randn(rng, Float32, (200, 300))
;

Calculate the number of single precision floating point operations involved in this calculation:

In [16]:
numspflops = *(size(t1)[2:end]..., size(t2)[2:end]...) * (2 * size(t1)[1] - 1)

21546000000

### CPU single threaded approach

In [17]:
t3 = Array{Float32}(undef, size(t1)[2], size(t1)[3], size(t2)[2])

function calc_t3_loop!(t3, t1, t2)
    for k in 1:size(t3)[3]
        for j in 1:size(t3)[2]
            for i in 1:size(t3)[1]
                @inbounds t3[i,j,k] = t1[1,i,j] * t2[1,k]

                for l in 2:size(t1)[1]
                    @inbounds t3[i,j,k] += t1[l,i,j] * t2[l,k]
                end
            end
        end
    end
end

trial = @benchmark calc_t3_loop!($t3, $t1, $t2)
print("GFLOPS (SP): $(numspflops/median(trial).time)\n")
@show trial

GFLOPS (SP): 2.275835175870624
trial = Trial(9.467 s)


BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     9.467 s (0.00% GC)
  median time:      9.467 s (0.00% GC)
  mean time:        9.467 s (0.00% GC)
  maximum time:     9.467 s (0.00% GC)
  --------------
  samples:          1
  evals/sample:     1

### CPU multi-threaded approach

In [18]:
t3_2 = Array{Float32}(undef, size(t1)[2], size(t1)[3], size(t2)[2])

function calc_t3_loop_threaded!(t3, t1, t2)
    Threads.@threads for k in 1:size(t3)[3]
        for j in 1:size(t3)[2]
            for i in 1:size(t3)[1]
                @inbounds t3[i,j,k] = t1[1,i,j] * t2[1,k]

                for l in 2:size(t1)[1]
                    @inbounds t3[i,j,k] += t1[l,i,j] * t2[l,k]
                end
            end
        end
    end
end

trial = @benchmark calc_t3_loop_threaded!($t3_2, $t1, $t2)
@test maximum(abs.(t3_2 - t3)) .< 1e-5 # confirm results match the single threaded results
print("GFLOPS (SP): $(numspflops/median(trial).time)\n")
@show trial

GFLOPS (SP): 55.97422159779707
trial = Trial(319.334 ms)


BenchmarkTools.Trial: 
  memory estimate:  48 bytes
  allocs estimate:  1
  --------------
  minimum time:     319.334 ms (0.00% GC)
  median time:      384.927 ms (0.00% GC)
  mean time:        367.775 ms (0.00% GC)
  maximum time:     401.112 ms (0.00% GC)
  --------------
  samples:          14
  evals/sample:     1

### Single GPU

In [26]:
t1cu = CuArray(t1)
t2cu = CuArray(t2)
t3cu = CuArray{Float32}(undef, size(t1)[2], size(t1)[3], size(t2)[2])

function calc_t3_gpu!(t3, t1, t2)
    function kernel!(t3, t1, t2)
        xindex = threadIdx().x + (blockIdx().x - 1)*blockDim().x
        yindex = threadIdx().y + (blockIdx().y - 1)*blockDim().y
        zindex = threadIdx().z + (blockIdx().z - 1)*blockDim().z
        
        xstride = blockDim().x * gridDim().x
        ystride = blockDim().y * gridDim().y
        zstride = blockDim().z * gridDim().z
    
        for k in zindex:zstride:size(t3)[3]
            for j in yindex:ystride:size(t3)[2]
                for i in xindex:xstride:size(t3)[1]
                    @inbounds t3[i,j,k] = t1[1,i,j] * t2[1,k]
                    
                    for l in 2:size(t1)[1]
                        @inbounds t3[i,j,k] += t1[l,i,j] * t2[l,k]
                    end
                end
            end
        end
        
        return nothing
    end

    numthreads = (4,4,4)
    numblocks = map(x -> ceil(Int, x), (size(t3) ./ numthreads))
    CuArrays.@sync begin
        @cuda threads=numthreads blocks=numblocks kernel!(t3, t1, t2)
    end
end

trial = @benchmark calc_t3_gpu!($t3cu, $t1cu, $t2cu)
@test maximum(abs.(Array(t3cu) - t3)) < 1e-4 # confirm results match the single threaded results
print("GFLOPS (SP): $(numspflops/median(trial).time)\n")
@show trial

GFLOPS (SP): 199.42954228133993
trial = Trial(107.263 ms)


BenchmarkTools.Trial: 
  memory estimate:  528 bytes
  allocs estimate:  15
  --------------
  minimum time:     107.263 ms (0.00% GC)
  median time:      108.038 ms (0.00% GC)
  mean time:        107.975 ms (0.00% GC)
  maximum time:     108.270 ms (0.00% GC)
  --------------
  samples:          47
  evals/sample:     1

### Multiple GPUs

In [20]:
function allocgpuarrays(t1, t2, numgpus)
    t1s = Array{CuArray{Float32,3}}(undef,0)
    t2s = Array{CuArray{Float32,2}}(undef,0)
    t3s = Array{CuArray{Float32,3}}(undef,0)
    slabsize = ceil(Int, size(t1)[2]/numgpus)
    for i in 1:numgpus
        indexstart = (i-1)*slabsize + 1
        indexend = min(i*slabsize, size(t1)[2])
        device!(i-1) do
            push!(t1s, CuArray(t1[:,indexstart:indexend,:]))
            push!(t2s, CuArray(t2))
            push!(t3s, CuArray{Float32}(undef, indexend - indexstart + 1, size(t1)[3], size(t2)[2]))
        end
    end
    
    return t1s, t2s, t3s
end

t1s, t2s, t3s = allocgpuarrays(t1, t2, numgpus);

In [24]:
function calc_t3_gpu_nosync!(t3, t1, t2)
    function kernel!(t3, t1, t2)
        xindex = threadIdx().x + (blockIdx().x - 1)*blockDim().x
        yindex = threadIdx().y + (blockIdx().y - 1)*blockDim().y
        zindex = threadIdx().z + (blockIdx().z - 1)*blockDim().z
        
        xstride = blockDim().x * gridDim().x
        ystride = blockDim().y * gridDim().y
        zstride = blockDim().z * gridDim().z
    
        for k in zindex:zstride:size(t3)[3]
            for j in yindex:ystride:size(t3)[2]
                for i in xindex:xstride:size(t3)[1]
                    @inbounds t3[i,j,k] = t1[1,i,j] * t2[1,k]
                    
                    for l in 2:size(t1)[1]
                        @inbounds t3[i,j,k] += t1[l,i,j] * t2[l,k]
                    end
                end
            end
        end
        
        return nothing
    end

    numthreads = (8,4,4)
    numblocks = map(x -> ceil(Int, x), (size(t3) ./ numthreads))
    @cuda threads=numthreads blocks=numblocks kernel!(t3, t1, t2)
end
;

In [25]:
function calc_t3_gpus!(t3s, t1s, t2s)
    for i in 1:numgpus
        device!(i-1) do
            calc_t3_gpu_nosync!(t3s[i], t1s[i], t2s[i])
        end
    end
    # resynchronize each device (for timing purposes)
    for i in 1:numgpus
        device!(i-1) do
            CuArrays.@sync 1
        end
    end
end

trial = @benchmark calc_t3_gpus!($t3s, $t1s, $t2s)
@test maximum(abs.(vcat(map(Array,t3s)...) - t3)) < 1e-4 # confirm results match the single threaded results
print("GFLOPS (SP): $(numspflops/median(trial).time)\n")
@show trial

GFLOPS (SP): 1724.17518311259
trial = Trial(12.366 ms)


BenchmarkTools.Trial: 
  memory estimate:  7.56 KiB
  allocs estimate:  262
  --------------
  minimum time:     12.366 ms (0.00% GC)
  median time:      12.496 ms (0.00% GC)
  mean time:        12.658 ms (0.00% GC)
  maximum time:     15.244 ms (0.00% GC)
  --------------
  samples:          395
  evals/sample:     1